# Metrics!

Для свого курсового проекту визначте остаточні метрики і напишіть програму, яка їх реалізує. Покажіть приклад роботи програми на іграшкових даних (до 10 прикладів реальних чи штучних даних).


## Поділ на склади

* якщо між голосними звуками є один приголосний, то він належить до наступного складу: о-сінь, про-синь, Си-дір;
* якщо між голосними є кілька приголосних, то й, в, р, л, м, н, які стоять після голосного, належать до попереднього складу, а звуки, що йдуть після них, — до наступного: гай-ка, Вол-га, мір-ка;
* якщо другим приголосним виступають й, р, л, то разом з попереднім приголосним вони відходять до наступного складу: мі-льйон, лю-блю, Дми-тро;
* якщо між голосними є кілька приголосних, то після наголосу один з них відходить до попереднього, а решта — до наступного складу: мас-ло, eip-ний, рів-ний;
* якщо після ненаголошеного складу стоїть кілька приголосних, то всі вони, крім й, в, л, відходять до наступного: се-стра, пе-ре-рва-ти, про-сте-жи-ти.

In [ ]:
def syllable_break(s):
    
    s = s.lower()
    
    alphabet = set('абвгґдеєжзиіїйклмнопрстуфхцчшщьюя’')
    vow = set('аеиіеоуяєїєю')
    cons = set('бвгґджзйклмнпрстфхцчшщь')
    
    group1 = set("йврлмн")
    group2 = set('йрл')
    group3 = cons.symmetric_difference(set('йвл'))    
    
    breaks = []
    n = len(s)
    
    for i in range(0, n):
        
        """якщо між голосними звуками є один приголосний, то 
        він належить до наступного складу: о-сінь, про-синь, Си-дір;"""
        
        if i < n-2 and s[i] in vow and s[i+1] in cons and s[i+2] in vow:
            print('{0}-{1}{2}'.format(s[i], s[i+1], s[i+2]))
            breaks.append(i+1)
            
        """якщо між голосними є кілька приголосних, то й, в, р, л, м, н, 
        які стоять після голосного, належать до попереднього складу, а 
        звуки, що йдуть після них, — до наступного: гай-ка, Вол-га, мір-ка;"""
        
        if i < n-2 and s[i] in vow and s[i+1] in group1 and s[i+2] in cons:
            breaks.append(i+1)
            
        """якщо другим приголосним виступають й, р, л, то разом з попереднім 
        приголосним вони відходять до наступного складу: мі-льйон, лю-блю, 
        Дми-тро;"""
        
        if i < n-1 and s[i] in cons and s[i+1] in group2:
            breaks.append(i-1)
            
        """якщо між голосними є кілька приголосних, то після наголосу один 
        з них відходить до попереднього, а решта — до наступного складу: 
        мас-ло, вip-ний, рів-ний;"""
        
        """якщо після ненаголошеного складу стоїть кілька приголосних, то всі 
        вони, крім й, в, л, відходять до наступного: се-стра, пе-ре-рва-ти, 
        про-сте-жи-ти."""
        
    
    print(breaks)
    

    
syllable_break('люблю')

## Визначення наголосу

Щоб визначити, чи відповідає той чи інший рядок поетичному метру, а також щоб правильно розбивати слова на склади, нам необхідно реалізувати програму, яка буде коректно виставляти наголоси в українських словах.

Огляд джерел показує, що детерміністичних правил наголошення в українській мові немає. Натомість, існує декілька загальних вказівок, а також словники, де в словах вказується наголос (зокрема, [орфоепічний](http://irbis-nbuv.gov.ua/cgi-bin/ua/elib.exe?Z21ID=&I21DBN=UKRLIB&P21DBN=UKRLIB&S21STN=1&S21REF=10&S21FMT=online_book&C21COM=S&S21CNR=20&S21P01=0&S21P02=0&S21P03=FF=&S21STR=ukr0001233). Також наголоси наводяться для частини слів у [СУМі](http://sum.in.ua)).

Отже, пропонуємо отримати вихідні дані з СУМу, після чого на цих даних спробувати навчити ML-модель.

In [45]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import re
import json

from pprint import pprint as pp

import trident

class BannedError(Exception):
    pass




    
def dump(entries, target_file='dump.txt'):
    """Dump list of entries (words, urls) into a newline-delimited text file"""
    
    try:
        with open(target_file, 'w+') as f:
            for entry in entries:
                f.write(entry + '\n')
    except:
        return -1
    
    return 0


# Scraper code
def run_request(url):
    """Fetch & soupify url. Use sparingly"""
    
    # This will mask us as a legit user and help avoid 424
    headers = {
        'User-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15',
        'Referer': 'https://sum.in.ua/',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'br, gzip, deflate',
        'Host': 'sum.in.ua',
        'Connection': 'keep-alive'
    }
    
    ban_line = 'Вибачте, доступ до сайту тимчасово закрито.'
    
    session = trident.init()
    # session = requests.session(headers=headers)
    
    # Run request
    r = session.get(url, headers=headers)
    soup = bs(r.text, 'html.parser')
    
    if ban_line in soup.text:
        raise BannedError("You have been temporarily banned. Wait until the ban is removed or change your IP address.")
        trident.shift()
        
    # Return a BS4 object
    return soup


def get_word(url):
    """Scrape article url to obtain word stress information
    from sum.in.ua
    
    :rtype word - string with stressed vowel in uppercase e.g. 'safAri'
    or -1 if no stress information was obtained from target article"""
    
    soup = run_request(url)
    word = ''
    
    # Some articles don't have stress info, so this 
    # flag will indicate if stress info was scraped
    article_contains_stress = False
    
    word_tag = soup.find_all(class_='title')[0]

    for child in word_tag.children:
        if type(child) == bs4.element.NavigableString:
            word += child.lower()
        elif type(child) == bs4.element.Tag:
            class_ = child.get('class')[0]
            if class_ == 'stressed':
                word += child.text.upper()
                article_contains_stress = True
        else:
            pass
    
    # If no stress info was obtained from target 
    # article, return an error value
    if article_contains_stress:
        return word
    else: 
        return -1

def get_index(url):
    """Scrape letter index url to obtain list of words to scrape"""

    soup = run_request(url)    
    link_list = []
    
    # print(soup)
        
    for li in soup.find_all('li'):
        for a in li.children:
            href = a.get('href')
            link_list.append(href)
    
    return link_list
    

# Rules to idetify types of links encountered
def is_index(s):
    if len(re.findall(r'\/vkazivnyk\/', s)) > 0:
        return True
    else:
        return False

def is_entry(s):
    if len(re.findall(r'\/s\/', s)) > 0:
        return True
    else:
        return False


def scrape_indices_boot():
    base_url = 'http://sum.in.ua/vkazivnyk/'
    root_url = 'http://sum.in.ua'
    
    indices = PersistentIndex(visited_name='indices_visited.txt', 
                              unvisited_name='indices_unvisited.txt')
    
    entries = PersistentIndex(visited_name='entries_visited.txt',
                              unvisited_name='entries_unvisited.txt')
    
    links = get_index(base_url)
    
    indices, entries = update_indices(indices, entries, links)
    
    while len(indices.unvisited) > 0:
        for item in indices.unvisited:
            if not indices.is_visited(item):
                links = get_index(root_url + item)
                indices, entries = update_indices(indices, entries, links)
                indices.mark_as_visited(item)
            
    # print(indices.unvisited)        
    indices.save_state()
    entries.save_state()

def update_indices(indices, entries, links):
    """
    go into each index
    get subindex
    compare it with visited and unvisited
        add new unvisited
    
    return indices, entries, links
    
    """
    
    for item in indices.unvisited:
        
        # Caches are added to avoid changing set size during iteration
        indices_cache = set()
        entries_cache = set()
        
        for link in links:
            if is_index(link) and not indices.is_visited(link):
                indices_cache.add(link)
            if is_entry(link) and not entries.is_visited(link):
                entries_cache.add(link)
        
        indices.update_unvisited(indices_cache)
        entries.update_unvisited(entries_cache)
        
    return indices, entries
    
scrape_indices_boot()

RuntimeError: Set changed size during iteration

In [30]:
i.is_unvisited('index1')

False

In [ ]:
url = 'http://sum.in.ua/vkazivnyk/ab'





words = []
urls = get_index(url)

for word_url in urls:
    words.append(get_word('http://sum.in.ua/'+word_url))

print(words)

In [ ]:
soup.text

In [ ]:
import requests

session = requests.session()
session.proxies = {
    'http' : 'socks5h://localhost:9050',
    'https' : 'socks5h://localhost:9050'
}

r = session.get('http://httpbin.org/ip')
print(r.text)

In [1]:
import trident

session = trident.init()

r = session.get('http://httpbin.org/ip')
print(r.text)

{
  "origin": "38.117.96.154, 38.117.96.154"
}



In [8]:
trident.shift(session)

In [7]:
r = session.get('http://httpbin.org/ip')
print(r.text)

{
  "origin": "38.117.96.154, 38.117.96.154"
}



In [ ]:
4/9/19, 12:22:16.669 [NOTICE] Bootstrapped 10%: Finishing handshake with directory server 
4/9/19, 12:22:16.787 [NOTICE] Bootstrapped 15%: Establishing an encrypted directory connection 
4/9/19, 12:22:16.830 [NOTICE] Bootstrapped 20%: Asking for networkstatus consensus 
4/9/19, 12:22:16.876 [NOTICE] Bootstrapped 25%: Loading networkstatus consensus 
4/9/19, 12:22:17.735 [NOTICE] Bootstrapped 45%: Asking for relay descriptors 
4/9/19, 12:22:17.736 [NOTICE] I learned some more directory information, but not enough to build a circuit: We need more microdescriptors: we have 3904/6640, and can only build 30% of likely paths. (We have 67% of guards bw, 64% of midpoint bw, and 69% of exit bw = 30% of path bw.) 
4/9/19, 12:22:18.372 [NOTICE] Bootstrapped 64%: Loading relay descriptors 
4/9/19, 12:22:18.372 [NOTICE] Bootstrapped 70%: Loading relay descriptors 
4/9/19, 12:22:19.369 [NOTICE] Bootstrapped 76%: Loading relay descriptors 
4/9/19, 12:22:19.370 [NOTICE] Bootstrapped 80%: Connecting to the Tor network 
4/9/19, 12:22:19.371 [NOTICE] Bootstrapped 85%: Finishing handshake with first hop 
4/9/19, 12:22:19.372 [NOTICE] Bootstrapped 90%: Establishing a Tor circuit 
4/9/19, 12:22:19.373 [NOTICE] Bootstrapped 100%: Done 
4/9/19, 12:22:20.576 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:22:20.826 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:37:06.263 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:40:32.779 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:43:53.387 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:44:09.960 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:53:17.445 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:53:23.768 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:53:23.772 [NOTICE] Rate limiting NEWNYM request: delaying by 4 second(s) 
4/9/19, 12:53:30.228 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:53:30.236 [NOTICE] Rate limiting NEWNYM request: delaying by 7 second(s) 
4/9/19, 12:55:01.799 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:55:11.163 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 12:55:11.163 [NOTICE] Rate limiting NEWNYM request: delaying by 1 second(s) 
4/9/19, 12:58:21.690 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:00:10.573 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:08:42.471 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:08:53.414 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:09:52.294 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:09:59.238 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:09:59.242 [NOTICE] Rate limiting NEWNYM request: delaying by 3 second(s) 
4/9/19, 13:10:46.550 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:11:38.454 [NOTICE] New control connection opened from 127.0.0.1. 
4/9/19, 13:12:20.274 [NOTICE] New control connection opened from 127.0.0.1. 
